### 고객 만족 데이터 세트를 이용한 앙상블 실습

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# ensemble 위한 분류학습기 추가하기

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler , Binarizer 
from sklearn.model_selection import train_test_split , GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_curve , roc_curve

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris

from sklearn.datasets import load_breast_cancer
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [41]:
# 학습 데이터
customer_train = pd.read_csv('/Users/HojinChoi/Desktop/data/train.csv')
customer_train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [46]:
features = pd.read_csv('/Users/HojinChoi/Desktop/data/train.csv')
del features['TARGET']

features.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016


In [28]:
customer_test = pd.read_csv('/Users/HojinChoi/Desktop/data/test.csv')
customer_test.head()
# customer_test.info()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [35]:
# 클래스에 대한 비율 확인
print(customer_train['TARGET'].value_counts())

# 불만족 비율
unsati_cnt = customer_train[customer_train['TARGET'] == 1]['TARGET'].count() / customer_train['TARGET'].count()
print('unsatisfied ratio : ' , unsati_cnt)

0    73012
1     3008
Name: TARGET, dtype: int64
unsatisfied ratio :  0.0395685345961589


In [36]:
# [실습] XGBM을 이용한 예측
# 결측치 확인
# 성능 평가를 조기 중단, 파라미터를 설정하고 학습/예측/평가
# 하이퍼 파라미터 튜닝 - GridSearchCV - 교차 검증
# XGBM을 이용한 예측
# 변수 중요도 시각화
# 스태킹 모델로 변환해 성능 평가

In [47]:
X_train , X_test , y_train , y_test = train_test_split(features,
                                                                              customer_train['TARGET'],
                                                                              test_size=0.2,
                                                                              random_state=100)

In [48]:
from xgboost import XGBClassifier

In [70]:
sklearn_xgboost_model = XGBClassifier(n_estimators=400,
                                                                 learning_rate=0.1,
                                                                 max_depth=3)

In [76]:
sklearn_xgboost_model.fit(X_train , y_train ,
                                         early_stopping_rounds=100,
                                         eval_metric='logloss',  #오류에 대한 평가성능지표 리턴 함수
                                         eval_set = [(X_train, y_train), (X_test , y_test)],
                                         verbose = True)

[0]	validation_0-logloss:0.61151	validation_1-logloss:0.61177
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.54471	validation_1-logloss:0.54518
[2]	validation_0-logloss:0.48919	validation_1-logloss:0.48986
[3]	validation_0-logloss:0.44246	validation_1-logloss:0.44329
[4]	validation_0-logloss:0.40280	validation_1-logloss:0.40375
[5]	validation_0-logloss:0.36882	validation_1-logloss:0.36995
[6]	validation_0-logloss:0.33959	validation_1-logloss:0.34085
[7]	validation_0-logloss:0.31433	validation_1-logloss:0.31570
[8]	validation_0-logloss:0.29242	validation_1-logloss:0.29389
[9]	validation_0-logloss:0.27334	validation_1-logloss:0.27489
[10]	validation_0-logloss:0.25670	validation_1-logloss:0.25832
[11]	validation_0-logloss:0.24212	validation_1-logloss:0.24386
[12]	validation_0-logloss:0.22937	validation_1-logloss:0.23117
[13]	validation_0-logloss

[128]	validation_0-logloss:0.12851	validation_1-logloss:0.13537
[129]	validation_0-logloss:0.12849	validation_1-logloss:0.13537
[130]	validation_0-logloss:0.12845	validation_1-logloss:0.13534
[131]	validation_0-logloss:0.12839	validation_1-logloss:0.13534
[132]	validation_0-logloss:0.12835	validation_1-logloss:0.13533
[133]	validation_0-logloss:0.12829	validation_1-logloss:0.13534
[134]	validation_0-logloss:0.12828	validation_1-logloss:0.13532
[135]	validation_0-logloss:0.12825	validation_1-logloss:0.13531
[136]	validation_0-logloss:0.12823	validation_1-logloss:0.13533
[137]	validation_0-logloss:0.12821	validation_1-logloss:0.13532
[138]	validation_0-logloss:0.12817	validation_1-logloss:0.13532
[139]	validation_0-logloss:0.12816	validation_1-logloss:0.13531
[140]	validation_0-logloss:0.12812	validation_1-logloss:0.13533
[141]	validation_0-logloss:0.12810	validation_1-logloss:0.13531
[142]	validation_0-logloss:0.12806	validation_1-logloss:0.13533
[143]	validation_0-logloss:0.12805	valid

[257]	validation_0-logloss:0.12465	validation_1-logloss:0.13529
[258]	validation_0-logloss:0.12459	validation_1-logloss:0.13530
[259]	validation_0-logloss:0.12456	validation_1-logloss:0.13529
[260]	validation_0-logloss:0.12454	validation_1-logloss:0.13530
[261]	validation_0-logloss:0.12452	validation_1-logloss:0.13531
[262]	validation_0-logloss:0.12450	validation_1-logloss:0.13531
[263]	validation_0-logloss:0.12446	validation_1-logloss:0.13534
[264]	validation_0-logloss:0.12440	validation_1-logloss:0.13536
[265]	validation_0-logloss:0.12435	validation_1-logloss:0.13535
[266]	validation_0-logloss:0.12431	validation_1-logloss:0.13537
[267]	validation_0-logloss:0.12426	validation_1-logloss:0.13538
[268]	validation_0-logloss:0.12425	validation_1-logloss:0.13539
[269]	validation_0-logloss:0.12422	validation_1-logloss:0.13538
[270]	validation_0-logloss:0.12421	validation_1-logloss:0.13539
[271]	validation_0-logloss:0.12417	validation_1-logloss:0.13540
[272]	validation_0-logloss:0.12414	valid

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [72]:
y_pred = sklearn_xgboost_model.predict(X_test)

In [73]:
# 혼동행렬 , 정확도 , 정밀도 , 재현율 , F1 , AUC 확인까지

def classifier_eval(y_test , y_pred) :
    print('오차행렬\ : ' , confusion_matrix(y_test, y_pred))
    print('정확도 : ' , accuracy_score(y_test, y_pred))
    print('정밀도 : ' , precision_score(y_test, y_pred))
    print('재현율 : ' , recall_score(y_test, y_pred))
    print('F1 : ' , f1_score(y_test, y_pred))
    print('AUC : ' , roc_auc_score(y_test, y_pred))

In [74]:
classifier_eval(y_test , y_pred)

오차행렬\ :  [[14584     1]
 [  616     3]]
정확도 :  0.9594185740594581
정밀도 :  0.75
재현율 :  0.004846526655896607
F1 :  0.009630818619582664
AUC :  0.5023889815315822


In [75]:
# GridSearchCV 교차 검증 및 하이퍼 파라미터 튜닝
params = {
    'n_estimators' : [100] , 
    'learning_rate' : [0.1 , 0.2 , 0.3] , # 랜포는 기본 depth 가 5 이므로 6부터 세팅
    'max_depth' : [1 , 2 , 3]
}

grid_cv = GridSearchCV(sklearn_xgboost_model , param_grid=params , cv=5 , n_jobs=-1)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=nan, monotone_constraints='()',
                                     n_estimators=400, n_jobs=0,
                                     num_parallel_tree=1, random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact', validate_parameters=1,
                                     verbosity=None),
